In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/Sollen-R/DS300-Final/main/Datasets/initial_dataset.csv?token=GHSAT0AAAAAAB3TVBQ3IV4JETQGK7XYAC7AY4D7LBQ')

In [4]:
df.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 89 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   url                         11430 non-null  object 
 1   length_url                  11430 non-null  int64  
 2   length_hostname             11430 non-null  int64  
 3   ip                          11430 non-null  int64  
 4   nb_dots                     11430 non-null  int64  
 5   nb_hyphens                  11430 non-null  int64  
 6   nb_at                       11430 non-null  int64  
 7   nb_qm                       11430 non-null  int64  
 8   nb_and                      11430 non-null  int64  
 9   nb_or                       11430 non-null  int64  
 10  nb_eq                       11430 non-null  int64  
 11  nb_underscore               11430 non-null  int64  
 12  nb_tilde                    11430 non-null  int64  
 13  nb_percent                  114

In [6]:
df.drop('url', axis='columns', inplace=True)

In [7]:
useless_cols = []

for col in df:
    if len(df[col].value_counts()) == 1:
        useless_cols.append(col)
        
print(useless_cols)

['nb_or', 'ratio_nullHyperlinks', 'ratio_intRedirection', 'ratio_intErrors', 'submit_email', 'sfh']


In [8]:
df.drop(useless_cols, axis='columns', inplace=True)

In [28]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
ss = StandardScaler()
le = LabelEncoder()

df['status'] = le.fit_transform(df['status'])
df_scaled = pd.DataFrame(ss.fit_transform(df), columns=df.columns)

x = df_scaled.drop('status', axis='columns')
y = df['status']

In [29]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.2, random_state=8675309)

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [31]:
model = keras.Sequential(layers=[layers.Dense(60, 'relu', input_shape=[81]),
                                layers.Dense(60, 'relu'),
                                layers.Dense(60, 'relu'),
                                layers.Dense(1, 'sigmoid')],
                         name='final-project-nn')

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
hist = model.fit(x_train, y_train, batch_size=1000, epochs=128, validation_data=(x_test, y_test))

Epoch 1/128
3/3 [==============================] - 2s 366ms/step - loss: 0.6839 - accuracy: 0.5061 - val_loss: 0.6387 - val_accuracy: 0.6153
Epoch 2/128
3/3 [==============================] - 0s 64ms/step - loss: 0.6279 - accuracy: 0.6492 - val_loss: 0.5930 - val_accuracy: 0.7428
Epoch 3/128
3/3 [==============================] - 0s 52ms/step - loss: 0.5788 - accuracy: 0.7712 - val_loss: 0.5492 - val_accuracy: 0.8086
Epoch 4/128
3/3 [==============================] - 0s 44ms/step - loss: 0.5319 - accuracy: 0.8268 - val_loss: 0.5044 - val_accuracy: 0.8392
Epoch 5/128
3/3 [==============================] - 0s 44ms/step - loss: 0.4830 - accuracy: 0.8552 - val_loss: 0.4581 - val_accuracy: 0.8610
Epoch 6/128
3/3 [==============================] - 0s 48ms/step - loss: 0.4328 - accuracy: 0.8736 - val_loss: 0.4118 - val_accuracy: 0.8739
Epoch 7/128
3/3 [==============================] - 0s 44ms/step - loss: 0.3831 - accuracy: 0.8871 - val_loss: 0.3673 - val_accuracy: 0.8802
Epoch 8/128
3/3 [==

In [58]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = (model.predict(x_test) > .5).astype('int32')

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

286/286 [==============================] - 1s 3ms/step
[[4308  249]
 [ 282 4305]]
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      4557
           1       0.95      0.94      0.94      4587

    accuracy                           0.94      9144
   macro avg       0.94      0.94      0.94      9144
weighted avg       0.94      0.94      0.94      9144

